In [11]:
import pandas as pd
import os

def merge_department_responses(original_file, response_files, output_file):
    print(f"Loading original file: {original_file}")
    try:
        xl_original = pd.ExcelFile(original_file)
    except Exception as e:
        print(f"Error loading original file: {e}")
        return
    
    merged_sheets = {}
    
    for sheet in xl_original.sheet_names:
        print(f"Processing sheet: {sheet}")
        try:
            df_original = xl_original.parse(sheet)
        except Exception as e:
            print(f"Error reading sheet '{sheet}' from original file: {e}")
            continue
        
        print(f"Original sheet '{sheet}' loaded with {df_original.shape[0]} rows and {df_original.shape[1]} columns")
        
        for response_file in response_files:
            print(f"Checking response file: {response_file}")
            try:
                xl_response = pd.ExcelFile(response_file)
            except Exception as e:
                print(f"Error loading response file '{response_file}': {e}")
                continue
            
            if sheet in xl_response.sheet_names:
                print(f"Merging data from {response_file} for sheet: {sheet}")
                try:
                    df_response = xl_response.parse(sheet)
                    print(f"Response sheet '{sheet}' has {df_response.shape[0]} rows and {df_response.shape[1]} columns")
                    
                    new_columns = df_response.columns[len(df_original.columns):]
                    print(f"Identified new columns: {list(new_columns)}")
                    
                    df_original = df_original.merge(df_response[new_columns], left_index=True, right_index=True, how='left')
                    print(f"After merging, sheet '{sheet}' has {df_original.shape[0]} rows and {df_original.shape[1]} columns")
                except Exception as e:
                    print(f"Error merging sheet '{sheet}' from '{response_file}': {e}")
                    continue
        
        merged_sheets[sheet] = df_original
    
    print(f"Saving final merged file: {output_file}")
    try:
        with pd.ExcelWriter(output_file) as writer:
            for sheet, df in merged_sheets.items():
                df.to_excel(writer, sheet_name=sheet, index=False)
        print(f"Final merged Excel saved as {output_file}")
    except Exception as e:
        print(f"Error saving final merged file: {e}")

# Get user input for file paths
input_path = input("Enter the directory where Excel files are located: ").strip()
original_file_name = input("Enter the original Excel file name (including .xlsx): ").strip()
original_file = os.path.join(input_path, original_file_name)

# Ask user if all response files are in the same directory
auto_fetch = input("Should all response files be fetched automatically from this directory? (yes/no): ").strip().lower()
if auto_fetch == 'yes':
    response_files = [os.path.join(input_path, f) for f in os.listdir(input_path) if f.endswith(".xlsx") and f != original_file_name]
else:
    response_file_names = input("Enter the response Excel file names separated by commas: ").strip().split(',')
    response_files = [os.path.join(input_path, name.strip()) for name in response_file_names]

output_file = os.path.join(input_path, "final_merged.xlsx")

merge_department_responses(original_file, response_files, output_file)


Enter the directory where Excel files are located:  /Users/vineetdixit/Desktop/merge
Enter the original Excel file name (including .xlsx):  Audit Report May24-Sept24.xlsx
Should all response files be fetched automatically from this directory? (yes/no):  yes


Loading original file: /Users/vineetdixit/Desktop/merge/Audit Report May24-Sept24.xlsx
Processing sheet: Report
Original sheet 'Report' loaded with 34 rows and 5 columns
Checking response file: /Users/vineetdixit/Desktop/merge/Final Audit Report 3.xlsx
Merging data from /Users/vineetdixit/Desktop/merge/Final Audit Report 3.xlsx for sheet: Report
Response sheet 'Report' has 34 rows and 5 columns
Identified new columns: []
After merging, sheet 'Report' has 34 rows and 5 columns
Checking response file: /Users/vineetdixit/Desktop/merge/Final Audit Report 2.xlsx
Merging data from /Users/vineetdixit/Desktop/merge/Final Audit Report 2.xlsx for sheet: Report
Response sheet 'Report' has 34 rows and 5 columns
Identified new columns: []
After merging, sheet 'Report' has 34 rows and 5 columns
Checking response file: /Users/vineetdixit/Desktop/merge/final_merged.xlsx
Merging data from /Users/vineetdixit/Desktop/merge/final_merged.xlsx for sheet: Report
Response sheet 'Report' has 34 rows and 5 colu